# Spark Aufgaben
1. Importe laden
2. Jupyter Spark starten und Twitter-Streams von Avro lesen
3. ETL Strecke: Avro Daten einlesen und als Delta Datei wieder raus schreiben
4. Analyse-Aufgaben erledigen 
5. Verlaufsanalyse durchführen
6. **Ausschalten der Spark-App**

## Wichtige Hinweise
1. Führe alle Anweisungen in der vorgegebenen Reihenfolge aus. Die einzelnen Programmierzellen bauen aufeinander auf.
2. **Beende unbedingt am Ende die Spark-Anwendung mit dem untersten Befehl "spark.stop()" , wenn du aufhörst an den Daten zu arbeiten.**
3. Du kannst jederzeit das Notebook wieder hochfahren, wenn du Schritt 1 & 2 (Laden der Imports & Jupyter Spark und seine Konfigurationen hochfahren) ausführen.
4. Mit **"Strg" + "Enter"** führst du einzelne Zellen direkt aus.
5. In der oberen Leiste kannst du über **"Insert"** weitere Zellen hinzufügen, um weitere Test-Funktionen zu schreiben. 

## 1. Laden der Imports
Hier werden alle benötigten Libraries für dieses Lab heruntergeladen.

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.sql.functions import explode
from pyspark.sql.functions import lower, col
import pyspark.sql.functions as f

from delta import *


import datetime
from datetime import datetime
import json


# use 95% of the screen for jupyter cell
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important; }<style>"))

## 2. Jupyter Spark & Konfigurationen hochfahren
Hier wird die App jupyter-spark konfiguriert und hochgefahren, welche unsere weiteren Schritte ausführt.

In [ ]:
appName="jupyter-analytics"

conf = SparkConf()

# CLUSTER MANAGER
################################################################################
# set Kubernetes Master as Cluster Manager(“k8s://https://” is NOT a typo, this is how Spark knows the “provider” type).
conf.setMaster("k8s://https://kubernetes.default.svc.cluster.local:443")

# CONFIGURE KUBERNETES
################################################################################
# set the namespace that will be used for running the driver and executor pods.
conf.set("spark.kubernetes.namespace","frontend")
# set the docker image from which the Worker pods are created
conf.set("spark.kubernetes.container.image", "thinkportgmbh/workshops:spark-3.3.2")
conf.set("spark.kubernetes.container.image.pullPolicy", "Always")

# set service account to be used
conf.set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
# authentication for service account(required to create worker pods):
conf.set("spark.kubernetes.authenticate.caCertFile", "/var/run/secrets/kubernetes.io/serviceaccount/ca.crt")
conf.set("spark.kubernetes.authenticate.oauthTokenFile", "/var/run/secrets/kubernetes.io/serviceaccount/token")


# CONFIGURE SPARK
################################################################################
conf.set("spark.sql.session.timeZone", "Europe/Berlin")
# set driver host. In this case the ingres service for the spark driver
# find name of the driver service with 'kubectl get services' or in the helm chart configuration
conf.set("spark.driver.host", "jupyter-spark-driver.frontend.svc.cluster.local")
# set the port, If this port is busy, spark-shell tries to bind to another port.
conf.set("spark.driver.port", "29413")
# add the postgres driver jars into session
conf.set("spark.jars", "/opt/spark/jars/spark-avro_2.12-3.3.2.jar")
conf.set("spark.executor.extraClassPath","/opt/spark/jars/spark-avro_2.12-3.3.2.jar")
#conf.set("spark.executor.extraLibrary","/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /opt/spark/jars/kafka-clients-3.3.1.jar")
#conf.set("spark.driver.extraClassPath","/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /opt/spark/jars/kafka-clients-3.3.1.jar, /opt/spark/jars/spark-avro_2.12-3.3.1.jar")

# CONFIGURE S3 CONNECTOR
conf.set("spark.hadoop.fs.s3a.endpoint", "minio.minio.svc.cluster.local:9000")
conf.set("spark.hadoop.fs.s3a.access.key", "trainadm")
conf.set("spark.hadoop.fs.s3a.secret.key", "train@thinkport")
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

#conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension, org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions, org.apache.spark.sql.hudi.HoodieSparkSessionExtension")
conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# CONFIGURE WORKER (Customize based on workload)
################################################################################
# set number of worker pods
conf.set("spark.executor.instances", "1")
# set memory of each worker pod
conf.set("spark.executor.memory", "1G")
# set cpu of each worker pod
conf.set("spark.executor.cores", "2")
# Number of possible tasks = cores * executores

## Deltalake
# conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")

# SPARK SESSION
################################################################################
# and last, create the spark session and pass it the config object

spark = SparkSession\
    .builder\
    .config(conf=conf) \
    .config('spark.sql.session.timeZone', 'Europe/Berlin') \
    .appName(appName)\
    .getOrCreate()

# also get the spark context
sc=spark.sparkContext
# change the log level to warning, to see less output
sc.setLogLevel('ERROR')

# get the configuration object to check all the configurations the session was startet with
for entry in sc.getConf().getAll():
        if entry[0] in ["spark.app.name","spark.kubernetes.namespace","spark.executor.memory","spark.executor.cores","spark.driver.host","spark.master"]:
            print(entry[0],"=",entry[1])
            
spark

## 3. Einlesen und Schreiben von Daten

### 3.1 Einlesen der Avro Dateien von S3
Laden der Daten aus dem Bucket "s3a://twitter/avro" in einen DataFrame, um auf den Daten zu arbeiten. 

In [ ]:
df_avro=(spark
    .read.format("avro")
    # Pfad zu Bucket
    .load("s3a://twitter/avro")
    # repartition auf 20 um optimierter mit den wenigen cpu zu arbeiten
    .repartition(20)
   ).cache()


 # nur Tweets mit dem Hashtag BigData weiter verwenden
df = df_avro.filter(f.array_contains(f.col("hashtags"),"BigData")==True)

print("Anzahl aller Tweets: ",df_avro.count())
print("Anzahl Tweets mit BigData: ",df.count())

Kurz anschauen was da drin istErste Ausgabe der Daten in Form eines DataFrames

In [ ]:
print("Anzahl aller Tweets: ",df_avro.count())
print("Anzahl Tweets mit BigData: ",df.count())
df.show()

## 4. Analyse-Aufgaben


### 4.1 Tweets anschauen und den Aufbau des Dataframes
Schau dir den Datensatz einmal genau an. Welche Spalten gibt es? Welche Datentypen sind vorhanden?

In [ ]:
df.<<EIGENER CODE>>

### 4.2  Das Schema des Datensatzes anzeigen 
<br>
<code> df.printSchema()</code> gibt das Schema des Datensatzes aus.

In [ ]:
df.printSchema()

### 4.3 Zählen der Tweets pro Stunde
Schreibe eine Abfrage, die **die Anzahl an Tweets pro Stunde** zählt.  
Hilfreiche Dokumentation: https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-groupby.html
<details hidden>
<summary> &#8964 Tipp </summary>
<code>df_hourly=(&ltdataFrame&gt
            .withColumn(&ltrename_column&gt, f.hour(f.col("&ltcolumn&gt")))
            .groupBy("&ltrename_column&gt")
            .&ltfunctionToCount()&gt
            .withColumnRenamed("count","total")
            .sort("&ltrename_codf_top_user=(df
                ...
                )
df_top_user.show()
lumn&gt")
          )
df_hourly.show(20)</code>
</details>
</p>

In [ ]:
df_hourly=(df  
            ...
          )

df_hourly.show(20)

### 4.4 Top 10 User nach Tweet-Anzahl
Schreibe eine Abfrage, die die **Top User** nach ihrer **Anzahl an Tweets** ausgibt. Bedenke dabei, deine Ausgabe auf **10** Einträge zu limitieren.
<details hidden>
<summary> &#8964 Tipp</summary>

<code>df_top_user=(&ltdataFrame&gt
                .groupBy("&ltcolumnA&gt")
                .agg(
                    f.count("&ltcolumnA&gt").alias("numberOfTweets")
                    )
                .orderBy(f.col("&ltaggregatedColumn&gt").desc())
                .&ltfunctionTolimit(10)&gt
                )
df_top_user.show()</code>
</details>


In [ ]:
df_top_user=(df
                ...
                )
df_top_user.show()


### 4.5 Umgang mit Arrays
Für die folgenden Aufgabe wird die <code>explode</code>-Funktion benötigt. Schreibe eine Abfrage die das Hashtag-array mit <code>explode</code> teilt. Gebe dabei die Spalten "user_name", "tweet_id"und die explodierte"hashtags"- Spalte mit einem Limit von 20 Zeilen aus. 

https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.sql.functions.explode.html
<details hidden>
<summary> &#8964 Tipp </summary>
<p>
<code>df_hash=(df
         .withColumn("&ltcolumnC&gt",explode("&ltcolumnC&gt"))
        .limit(20)
        .select("&ltcolumnA&gt", "&ltcolumnB&gt", "&ltcolumnC&gt")
        )
df_hash.show()</code>
</details>
</p>

In [ ]:
df_hash=(df
         ...
        )
df_hash.show()

### 4.6 Top 5 Hashtags der Top 10 User
Schreibe eine Abfrage, die die **Top 5 der Hashtags** der **10 User** mit den **meisten Tweets** ausgibt.
<br>
<br>
<details>
<summary> &#8964 Hinweis </summary>
<p>
<code>df_top5_per_user=(&ltdataFrameA&gt
            # filter via join
            .join(&dataFrameBC&gt,[&ltdataFrameA&gt.&ltcolumnA&gt==&ltDataFrameB&gt.&ltcolumnB&gt],how="left")
            # hashtags array in Zeilen Einträge exploden
            .withColumn("&ltcolumnC&gt",explode("&ltcolumnC&gt"))
            # hashtags lowercase schreiben um Doppelungen zu entfernen
            .withColumn("&ltcolumnC&gt", lower(col('&ltcolumnC&gt')))
            # groupieren und counten by hashtag
            .&ltfunctionToGroup&gt("&ltcolumnC&gt").agg(f.count("&ltcolumnC&gt"))
            # rückwärts sortieren
            .&ltfunctionToSort&gt(f.col("count(&ltcolumnC&gt)").desc())
            # top 5 selectieren
            .limit(5) 
                 )
df_top5_per_user.show()</code>
</details>
</p>

In [ ]:
df_top5_per_user=(df_top_user
                ...
                 )
df_top5_per_user.show()

 ### 4.7 Top 10 Influencer (User mit #BigData-tweets mit den meisten Followern) 
 Schreibe eine Abfrage, die die **Top 10 Influencer** mit den **meisten Follower** zählt und sortiert anzeigt.
 <br>
<br>
<details>
<summary> &#8964 Hinweis </summary>
<p>
<code>df_top_influencer=(df
                .groupBy("&ltcolumnA&gt")
                .agg(
                    f.&ltfunctionForMaximum&gt("&ltcolumnB&gt").alias("follower")
                    )
                .&ltfunctionToOrder&gt(f.col("follower").desc())
                )
df_top_influencer.show(10)</code>
</details>
</p>

In [ ]:
df_top_influencer=(df
                ...
                )
df_top_influencer.show(10)

### 4.8 Top 10 Influencer und ihre Anzahl an tweets
Schreibe eine Abfrage, die die **Top 10 Influencer**, ihre Follower und die **Anzahl ihrer Tweets** ausgibt. außeredem soll es sortiert nach den Anzahl ihrer Follower sein. 
<br>
<br>
<details>
<summary> &#8964 Hinweis </summary>
<p>
<code>df_withRetweets=(&ltdataFrameA&gt
            # filter via join auf die Top 10 Influencer
            .join(&ltdataFrameB&gt, [&ltdataFrameA&gt.&ltcolumnA&gt==&ltdataFrameB&gt.&ltcolumnB&gt],how="&lthowToJoin&gt")
            .orderBy(f.col("&ltcolumnC&gt").desc())
            .limit(10)
            .drop("&ltdoubledColumn&gt")
            .select("&ltcolumnA&gt","&ltcolumnC&gt","&ltcolumnD&gt")
    )
df_withRetweets.show()</code>
</details>
</p>

In [ ]:
df_withRetweets=(df_top_user
            ....
    )

df_withRetweets.show()

### Bonusaufgabe: Filter nach den Top 10 Locations und ihrem Top Hashtag
Schreibe eine Abfrage, die die **Top 10 häufigsten Locations** ausgibt und das am **zweitmeisten verwendete Hashtag** dort. Da alle unsere Daten das Hashtag #BigData beinhalten. 
<br>
<br>
<details>
<summary> &#8964 Hinweis </summary>
<p>
<code>df3=(df
    .select("&ltcolumnA&gt")
    .where(~f.col("&ltcolumnC&gt").isin(<FilterWord1,...,FilterWordN>))
    .groupBy("&ltcolumnC&gt")
    .&ltfunctionToCount()&gt
    .withColumnRenamed("count","location_total")
    .orderBy(f.col("location_total").desc())
    .limit(10)
    )</code>
    
<code>df4=(df
    .select("&ltcolumnA&gt","&ltcolumnB&gt")
    .withColumn("singletag",f.explode(f.col("&ltcolumnB&gt")))
    .g&ltfunctiontoGroup&gt("&ltcolumnA&gt","singletag")
    .&ltfunctiontoCount()&gt
    .withColumnRenamed("count","tags_total")
        )</code>
    
<code>df5=(&ltdataFrameA&gt.alias("a")
    .&ltfunctiontoJoin&gt(f.broadcast(&ltdataFrameB&gt.alias("b")),[&ltdataFrameB&gt.&ltcolumnA&gt==&ltdataFrameB&gt.&ltcolumnB&gt],how="left")
    .select("a.&ltcolumnB&gt","a.location_total","b.singletag","b.tags_total")      
    .withColumn("rank",f.row_number().over(Window.partitionBy("a.&ltcolumnB&gt")
    .&ltfunctionToOrder&gt(f.col("b.tags_total").desc())))
    .&ltfunctionToFilter&gt(f.col("rank")==2)
    .&ltfunctionToSort&gt(f.col("location_total").&ltdescending&gt())
    .limit(10)
    )
df5.show()</code>
</details>
</p>

# 6. Ausschalten der Spark-App
**Bitte schließe am Ende die Spark-App wieder mit dem folgenden Befehl `spark.stop()`, wenn du fertig mit der Bearbeitung der Aufgaben bist.** 

In [ ]:
spark.stop()